<a href="https://colab.research.google.com/github/wardla2/Data-Science-Portfolio/blob/main/Project_2/2510_2_LWard_Project_2_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Home Sale Prices: A Data-Driven Approach to Real Estate Valuation

## Problem Definition


This goal of this project is to help a real estate company accurately predict house sale prices based on property features such as size, location, and age. The business goal is to improve pricing strategies and decision-making for buyers and sellers. This is a supervised regression problem, where the target variable is the numeric sale price. The  model will attempt minimize Root Mean Squared Percentage Error (RMSPE) to ensure accurate predictions.

## Data Collection/Sources


The data set contains information from the
Assessor's Office used in computing assessed values for individual
residential properties sold from 2006 to 2010.
It is hosted in an AWS S3 bucket.
https://www.google.com/url?q=https%3A%2F%2Fddc-datascience.s3.amazonaws.com%2FProjects%2FProject.2-Housing%2FData%2FHousing.Data.csv

## File Paths

In [ ]:
import pandas as pd

url = "https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv"


In [ ]:
# Additional imports

import numpy as np         # Numerical operations
import matplotlib.pyplot as plt    # Basic plotting
import seaborn as sns              # Statistical plots
import plotly.express as px        # Interactive plots

from sklearn.model_selection import train_test_split       # Splitting data
from sklearn.preprocessing import RobustScaler           # Feature scaling

from sklearn.preprocessing import OneHotEncoder            # Categorical encoding

from sklearn.metrics import mean_squared_error, r2_score   # Basic metrics


In [ ]:
# Load data
housing_df = pd.read_csv(url)
housing_df


## Data Cleaning


### Backup 1

In [ ]:
# Initial backup copy of the data frame

housing_df_bak1 = housing_df.copy()
housing_df_bak1


In [ ]:
housing_df.info()


In [ ]:
housing_df.describe()


In [ ]:
housing_df.shape


In [ ]:
housing_df.size


In [ ]:
housing_df.head(5)


In [ ]:
housing_df.tail(5)


In [ ]:
# View null values

housing_df.isna().sum()


In [ ]:
# Count the number of distinct elements in rows

housing_df.nunique().sort_values(ascending = False).head(20)


In [ ]:
housing_df.nunique().sort_values(ascending = False).tail(20)


In [ ]:
# Check if there are null series

housing_df.isnull().all()


In [ ]:
# Count nulls per column, sort descending

column_nulls = housing_df.isnull().sum().sort_values(ascending=False)

column_nulls


In [ ]:
column_nulls[column_nulls > 0]


In [ ]:
# Drop data series that have null values > ~ 1/3

drop = column_nulls[column_nulls > 880]
drop


In [ ]:
# Drop unecessary and low-value data series

housing_df.drop(columns=list(drop.index) + ['PID'], inplace=True)


In [ ]:
# Check remaining number of columns
housing_df.info()


In [ ]:
# Check for rows with null values in the 'SalePrice' field

housing_df['SalePrice'].isna()


In [ ]:
# Check for rows with >= 10% null values

row_nulls = housing_df[housing_df.isnull().sum(axis=1) >= 7]
row_nulls

In [ ]:
# Drop rows with >= 7 missing values

housing_df.drop(index=row_nulls.index, inplace=True)


In [ ]:
housing_df.info()


In [ ]:
housing_df.isna().sum().sort_values(ascending=False).head(20)


### Make backup 2

In [ ]:
# Make backup 2

housing_df_lowNaN = housing_df.copy()
housing_df_lowNaN


### Keep only numeric series

In [ ]:
# Identify numeric series

housing_numeric_series_df = housing_df_lowNaN.select_dtypes(include='number')
housing_numeric_series_df


In [ ]:
# Establish drop thresholds for series with significant zero values
zero_threshold = 0.55   # Drop if 55%+ values are zero

# Establish drop thresholds for series with low abs correlation with SalesPrice
corr_threshold = 0.45 #Drop if abs(correlation with SalesPrice) =<45%

# Calculate percent of zeros
zero_fraction = (housing_numeric_series_df == 0).sum() / len(housing_numeric_series_df)

# Correlation with target
correlations = housing_numeric_series_df.corr()['SalePrice'].abs()

# Identify series to drop
drop_numeric_series = zero_fraction[(zero_fraction > zero_threshold) & (correlations < corr_threshold)].index.tolist()
drop_numeric_series


In [ ]:
# Drop series that have significant zero values and and low abs correlation with SalesPrice

housing_numeric_series_df.drop(columns=drop_numeric_series , inplace=True)
housing_numeric_series_df


In [ ]:
# Look for numeric series correlated with each other & != target

# Compute feature correlation matrix
corr_matrix = housing_numeric_series_df.corr().abs()

# Get features' correlation with target
target_col = 'SalePrice'

# Make a list of correlated feature pairs
corr_threshold = 0.75

corr_pairs = [
  (col1, col2 , corr_matrix.loc[col1, col2])
  for col1 in corr_matrix.columns
  for col2 in corr_matrix.columns
  if col1 != col2
  and col1 != target_col
  and col2 != target_col
  and corr_matrix.loc[col1, col2] > corr_threshold
]

corr_pairs


In [ ]:
#  Get correlation of all features with SalePrice
target_corr = corr_matrix[target_col]
target_corr


In [ ]:
# Drop features less correlated with SalePrice

drop_features = set()

for col1, col2, corr_val in corr_pairs:
    if col1 in drop_features or col2 in drop_features:
        continue  # skip already selected for drop to avoid overlapping pairs
    if target_corr[col1] < target_corr[col2]:
        drop_features.add(col1)
    else:
        drop_features.add(col2)

# Then drop them from the DataFrame
housing_numeric_series_df.drop(columns=drop_features, inplace=True)


In [ ]:
housing_numeric_series_df.info()


In [ ]:
# Check if numeric dtypes series are continuous, nominal, or ordinal categorical in data dictionary:
# https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Housing%20-%20Data%20Documentation.pdf
drop_noncontinuous = ['MS SubClass','Overall Qual','Overall Cond','Mo Sold']

# Drop non-continuous series
housing_numeric_series_df.drop(columns = drop_noncontinuous, inplace=True)
housing_numeric_series_df


In [ ]:
housing_numeric_series_df.info()


In [ ]:
# Strip SalePrice (target var) and Yr Sold (possibly a numeric categorical) from data frame

housing_numeric_series_df.drop(columns=['Yr Sold'], inplace=True)
housing_numeric_series_df.info()


### Encode 'object' categorical series

In [ ]:
# These are the numeric categoricals identified in the numeric datatype series

numeric_categoricals = ['MS SubClass','Mo Sold','Yr Sold']

In [ ]:
# Select datatype 'object' series from DataFrame
categorical_obj = housing_df_lowNaN.select_dtypes(include='object')


# Build a summary DataFrame for 'object' data
obj_summary = pd.DataFrame({
    'column': categorical_obj.columns,
    'n_unique': [housing_df_lowNaN[col].nunique() for col in categorical_obj.columns]
})

# Sort summary DataFrame by number of unique values
obj_summary.sort_values(by='n_unique', ascending=True, inplace=True)

# Show
obj_summary.reset_index(drop=True, inplace=True)
obj_summary


In [ ]:
# Put categorical object columns in a list

categorical_obj = housing_df_lowNaN.select_dtypes(include='object').columns.tolist()

# Print unique values for each
for col in categorical_obj:
    print(f"{col} ({housing_df_lowNaN[col].nunique()} unique values):")
    print(housing_df_lowNaN[col].unique())
    print('-' * 15)


In [ ]:
# Check if numeric dtypes series are nominal or ordinal categorical in data dictionary:
# https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Housing%20-%20Data%20Documentation.pdf

# Make ordinal series list

obj_ordinal_series = ['Overall Qual','Overall Cond','Land Slope','Lot Shape','Utilities','Exter Qual',
                  'Exter Cond','Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin Type 2',
                  'Heating QC','Electrical','Kitchen Qual','Garage Finish','Garage Qual','Garage Cond','Paved Drive']
obj_ordinal_series


In [ ]:
# Make nominal series list

obj_nominal_series = obj_nominal_series = [col for col in categorical_obj if col not in obj_ordinal_series]
obj_nominal_series


### Plan for adding 'object' categorical series to prediction models

* One-hot encode nominal series and correlate with target series;
* Drop nominal series uncorrelated with target;
* Encode ord series as ordered numbers;
* Drop ord series uncorrelated with target;
* Concat numeric / nom / ord categorical series --> make all features DataFrame for train-test sets and model prediction






## Exploratory Data Analysis


### Independence of features & correlation

### Backup 3

In [ ]:
# Make simple var backup for numeric series EDA

housing_numeric_series_df_bak = housing_numeric_series_df.copy()
housing_df_eda1 = housing_numeric_series_df.copy()
housing_df_eda1


In [ ]:
# Check for skewness of numeric data series prior to scaling

skewed_vars = [
    col for col in housing_df_eda1.select_dtypes(include='number').columns
    if abs(housing_df_eda1[col].skew()) > 0.05
]
skewed_vars

In [ ]:
# Visually inspect numeric data series using KDE

# Select only numeric columns (exclude 'SalePrice')
numeric_cols = housing_df_eda1.select_dtypes(include='number').columns.drop('SalePrice', errors='ignore')

for col in numeric_cols:
    plt.figure(figsize=(5, 3))
    sns.histplot(data=housing_df_eda1, x=col, kde=True, bins=30, color='pink')
    plt.title(f'Distribution of {col}')
    plt.tight_layout()
    plt.show()


## Processing



### Scale numeric series

In [ ]:
# Set X (features) and y (target) values

X = housing_numeric_series_df.drop(columns='SalePrice')
y = housing_numeric_series_df['SalePrice']
X, y


In [ ]:
# Set train/test split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)
X_train, X_test, y_train, y_test


In [ ]:
# Fit the scaler on training data only

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled_array = X_train_scaled
X_train_scaled_array

In [ ]:
# Use the fitted scaler to transform test data

X_test_scaled = scaler.transform(X_test)
X_test_scaled_array = X_test_scaled
X_test_scaled_array


In [ ]:
# Make features_to_plot as list of column names
features_to_plot = X.columns

# Copy unscaled data
X_unscaled = X.copy()

# Convert scaled array back to DataFrame
X_scaled = pd.DataFrame(X_train_scaled_array, columns=features_to_plot)

# Plot
for col in features_to_plot:
    plt.figure(figsize=(10, 4))
    sns.kdeplot(X_unscaled[col], label='Unscaled', fill=True, linewidth=2)
    sns.kdeplot(X_scaled[col], label='Robust Scaled', fill=True, linewidth=2)
    plt.title(f"Distribution of '{col}' Before vs After Scaling")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


### Compare RMSPE for models applied to numeric series

In [ ]:
# Make RMSPE scorer
from sklearn.metrics import make_scorer

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

rmspe_scorer = make_scorer(rmspe, greater_is_better=False)


In [ ]:
# Impute NaN values and make recommended models

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Pipelines with imputer + scaler + model
models = {
    'Linear': make_pipeline(SimpleImputer(strategy='median'), RobustScaler(), LinearRegression()),
    'Ridge': make_pipeline(SimpleImputer(strategy='median'), RobustScaler(), Ridge(alpha=1.0)),
    'Lasso': make_pipeline(SimpleImputer(strategy='median'), RobustScaler(), Lasso(alpha=0.01, max_iter=10000))
}


In [ ]:
# Cross-validation

from sklearn.model_selection import cross_val_score

results = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, scoring=rmspe_scorer, cv=5)
    results[name] = -scores.mean()

# Results
for model_name, score in results.items():
  print(f"{model_name}: Mean RMSPE (CV=5) = {score:.5f}")


In [ ]:
# Isolate Ridge model

ridge_model = make_pipeline(
    SimpleImputer(strategy='median'),
    RobustScaler(),
    Ridge(alpha=1.0)
)

ridge_model.fit(X_train, y_train)


In [ ]:
# Inspect predictive features

from sklearn.linear_model import Ridge

# Access the trained Ridge model inside the pipeline
ridge_regressor = ridge_model.named_steps['ridge']

# Extract the coefficients
ridge_coefs = pd.Series(ridge_regressor.coef_, index=X_train.columns)

# Sort by absolute value
ridge_coefs_sorted = ridge_coefs.reindex(ridge_coefs.abs().sort_values(ascending=False).index)

# Show top features
print(ridge_coefs_sorted.head(15))


In [ ]:
# Plot predictive features

plt.figure(figsize=(8,6))
ridge_coefs_sorted.head(15).plot(kind='barh', color='steelblue')
plt.gca().invert_yaxis()  # largest at top
plt.title("Top 15 Most Predictive Features (Ridge Regression)")
plt.xlabel("Coefficient Magnitude")
plt.tight_layout()
plt.show()


### Test best prediction model (lowest Mean RMSPE)

In [ ]:
# Test prediction model

# Choose best model from cross-validation
final_model = models['Ridge']

# Fit on the entire training set
final_model.fit(X_train, y_train)

# Predict on the test data (scaling & imputation handled inside pipeline)
y_pred = final_model.predict(X_test)
y_pred #predicted sales prices


## Data Visualization/Communication of Results


In [ ]:
# Compare actual and predicted sales prices

comparison_df = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred
})

print(comparison_df.head(15))


In [ ]:
# Get mean RMSPE for test data set

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

test_rmspe = rmspe(y_test, y_pred)
print(f"Test data mean RMSPE (CV=5): {test_rmspe:.5f}")


In [ ]:
# Plot actual vs predicted prices for test data set

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # prediction line
plt.xlabel('Actual Sale Price')
plt.ylabel('Predicted Sale Price')
plt.title('Actual vs Predicted Sale Prices')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import scipy.stats as stats

# Compute residuals
residuals = y_test - y_pred

# Q-Q plot
plt.figure(figsize=(8,6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot of Residuals")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Residuals and fitted values

residuals = y_test - y_pred
fitted = y_pred

# Plot
plt.figure(figsize=(8,6))
plt.scatter(fitted, residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--', linewidth=2)
plt.xlabel("Fitted Values (Predicted Sale Price)")
plt.ylabel("Residuals (Actual - Predicted)")
plt.title("Residuals vs. Fitted Values")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Boxplot of residuals vs. year sold

x = housing_df_lowNaN['Yr Sold']

sns.boxplot(x=x, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Residuals by Year Sold")
plt.xlabel("Year Sold")
plt.ylabel("Residuals")
plt.tight_layout()
plt.show()


In [ ]:
# Residuals vs sales index - shows feature indepedence

res = residuals

plt.figure(figsize = (14,10))
plt.scatter(range(len(res)), res)
plt.plot(range(len(res)), res)
plt.title("Residuals by Sales Index")
plt.xlabel("Time")
plt.ylabel("Residuals")
plt.hlines(0, min(range(len(res))), max(range(len(res))), colors = 'red', linestyles = 'dashed') ;
